In [4]:
import os
import sys
import osmnx as ox
# Add the src folder to the Python path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)
from routing import get_nodes_on_road, get_roundabout_path

In [1]:
import ollama

def explain_route_with_llm(route_edges, instructions, model="llama3"):
    """
    Use an LLM to explain a route based on instructions and edge metadata.

    Parameters:
        route_edges (list[dict]): A list of edge dictionaries from OSMnx.
        instructions (list[str]): Navigation instructions step by step.
        model (str): Ollama model name, e.g., 'llama3' or 'mistral'.

    Returns:
        str: Natural language explanation from the LLM.
    """

    # Build prompt string
    prompt = "You are a helpful assistant describing a driving route in a city.\n\n"
    prompt += "You are given a sequence of navigation instructions and corresponding OpenStreetMap edge metadata.\n"
    prompt += "Your goal is to explain the route in plain English, summarizing key features like road names, turns, roundabouts, and any notable attributes.\n\n"

    prompt += "Instructions:\n"
    for i, instr in enumerate(instructions, 1):
        prompt += f"  Step {i}: {instr}\n"

    prompt += "\nRoute Edge Attributes:\n"
    for i, edge in enumerate(route_edges, 1):
        attrs = ", ".join(f"{k}: {v}" for k, v in edge.items() if k != "geometry")
        prompt += f"  Segment {i}: {attrs}\n"

    prompt += "\nNow write a clear explanation of the full route.\n"

    # LLM call
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": "You are an expert in road navigation and OpenStreetMap data."},
            {"role": "user", "content": prompt}
        ]
    )

    return response['message']['content']


In [2]:
import networkx as nx

def extract_route_edge_features(G: nx.MultiDiGraph, node_list: list) -> list:
    """
    Given a graph and a list of node IDs, extract descriptive edge features for LLM input.

    Returns:
        List[dict]: One dict per edge with key metadata fields.
    """
    edge_descriptions = []

    for u, v in zip(node_list[:-1], node_list[1:]):
        if not G.has_edge(u, v):
            raise ValueError(f"No edge between {u} and {v}")
        
        edge_data = G.get_edge_data(u, v)
        first_key = next(iter(edge_data))
        data = edge_data[first_key]

        desc = {
            "from_node": u,
            "to_node": v,
            "name": data.get("name", "unknown"),
            "highway": data.get("highway", "unknown"),
            "length_m": data.get("length", None),
            "oneway": data.get("oneway", False),
            "maxspeed": data.get("maxspeed", None),
            "lanes": data.get("lanes", None),
            "roundabout": data.get("junction") == "roundabout",
            "bridge": data.get("bridge", False),
            "tunnel": data.get("tunnel", False),
        }

        edge_descriptions.append(desc)

    return edge_descriptions

In [5]:
G = ox.load_graphml("../data/tolworth.graphml")

In [6]:
x = get_roundabout_path(G, 'Kingsdowne Road', 'Upper Brighton Road', 1736768194, 3)
edges = extract_route_edge_features(G, x)

In [ ]:
instructions = {'Step 1': {'C'}, 'Step 2': }

In [ ]:
explain_route_with_llm(edges, instructions, model="llama3.2:latest")